In [47]:
# create template for study folders
import synapseclient as sc
import pandas as pd
import numpy as np

syn = sc.login()


UPGRADE AVAILABLE

A more recent version of the Synapse Client (4.2.0) is available. Your version (3.2.0) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 4.2.0 release notes

https://python-docs.synapse.org/news/



Welcome, Nicholas Lee!



In [48]:
import os
from pathlib import Path
import re

In [49]:
cwd = Path(os.getcwd())

ROOT_DIR_NAME = "ELITE-data-models"

for p in cwd.parents:
    if bool(re.search(ROOT_DIR_NAME + "$", str(p))):
        print(p)
        ROOT_DIR = p

/Users/nlee/Documents/Projects/ELITE/ELITE-data-models


In [50]:
dm = pd.read_csv(Path(ROOT_DIR, 'EL.data.model.csv'))
dm_cols = list(dm.columns)
dm_cols

['Attribute',
 'Description',
 'Valid Values',
 'DependsOn',
 'DependsOn Component',
 'Required',
 'Parent',
 'Validation Rules',
 'Properties',
 'Source',
 'Module',
 'multivalue',
 'columnType',
 'Ontology',
 'Notes',
 'UsedIn']

In [51]:
cols = syn.getColumns("syn57394664")
cols = [c for c in cols]
study_temp = pd.DataFrame(cols)
# study_temp = study_temp.set_index("name")
study_temp = study_temp[['name', 'columnType']]
study_temp = study_temp[~study_temp['name'].isin(['id'])]
study_temp = study_temp.rename(columns={'name': 'Attribute'})
study_temp

,Attribute,columnType
1,studyKey,STRING
2,studyName,STRING
3,projectShortName,STRING
4,projectFullName,STRING
5,dataTypeAll,STRING_LIST
6,Species,STRING_LIST
7,specimenType,STRING_LIST
8,studyAbstract,STRING
9,studyDescription,STRING
10,studyFocus,STRING


In [52]:
for c in list(set(dm_cols) - set(study_temp.columns)): 
    study_temp[c] = None

study_temp = study_temp.assign(
    Required=True,
    Module="ManifestColumn",
    Properties="ManifestColumn",
    Source="Sage Bionetworks",
    Parent="ManifestColumn",
    UsedIn="study_folder_template",
    Ontology="Sage Bionetworks",
)

study_temp

,Attribute,columnType,UsedIn,DependsOn Component,DependsOn,Validation Rules,Ontology,Source,Notes,Required,Module,Valid Values,Properties,Description,multivalue,Parent
1,studyKey,STRING,study_folder_template,None,None,None,Sage Bionetworks,Sage Bionetworks,None,True,ManifestColumn,None,ManifestColumn,None,None,ManifestColumn
2,studyName,STRING,study_folder_template,None,None,None,Sage Bionetworks,Sage Bionetworks,None,True,ManifestColumn,None,ManifestColumn,None,None,ManifestColumn
3,projectShortName,STRING,study_folder_template,None,None,None,Sage Bionetworks,Sage Bionetworks,None,True,ManifestColumn,None,ManifestColumn,None,None,ManifestColumn
4,projectFullName,STRING,study_folder_template,None,None,None,Sage Bionetworks,Sage Bionetworks,None,True,ManifestColumn,None,ManifestColumn,None,None,ManifestColumn
5,dataTypeAll,STRING_LIST,study_folder_template,None,None,None,Sage Bionetworks,Sage Bionetworks,None,True,ManifestColumn,None,ManifestColumn,None,None,ManifestColumn
6,Species,STRING_LIST,study_folder_template,None,None,None,Sage Bionetworks,Sage Bionetworks,None,True,ManifestColumn,None,ManifestColumn,None,None,ManifestColumn
7,specimenType,STRING_LIST,study_folder_template,None,None,None,Sage Bionetworks,Sage Bionetworks,None,True,ManifestColumn,None,ManifestColumn,None,None,ManifestColumn
8,studyAbstract,STRING,study_folder_template,None,None,None,Sage Bionetworks,Sage Bionetworks,None,True,ManifestColumn,None,ManifestColumn,None,None,ManifestColumn
9,studyDescription,STRING,study_folder_template,None,None,None,Sage Bionetworks,Sage Bionetworks,None,True,ManifestColumn,None,ManifestColumn,None,None,ManifestColumn
10,studyFocus,STRING,study_folder_template,None,None,None,Sage Bionetworks,Sage Bionetworks,None,True,ManifestColumn,None,ManifestColumn,None,None,ManifestColumn


In [53]:
study_temp['multivalue'] = np.select(
    [
        study_temp['columnType'] == 'STRING_LIST'
    ],
    [
        True
    ],
    default = False
)

In [54]:
# add manifest attribute
new_manifest = [
    {
        "Attribute": "study_folder_template",
        "columnType": "NA",
        "Parent": "Component",
        "Module": "Template",
        "multivalue": "NA",
        "Description": "Annotations for study folders",
        "Properties": "Template",
        "DependsOn": ",".join(study_temp["Attribute"].tolist() + ["Component"]),
        "Source": "Sage Bionetworks",
        "Ontology": "Sage Bionetworks",
    }
]

new_manifest

# study_temp.loc[-1, "Attribute"] : "study_folder_annotations"
# study_temp.loc[-1,]

[{'Attribute': 'study_folder_template',
  'columnType': 'NA',
  'Parent': 'Component',
  'Module': 'Template',
  'multivalue': 'NA',
  'Description': 'Annotations for study folders',
  'Properties': 'Template',
  'DependsOn': 'studyKey,studyName,projectShortName,projectFullName,dataTypeAll,Species,specimenType,studyAbstract,studyDescription,studyFocus,program,grantNumber,studyWiki,acknowledgment,methods,relatedStudies,studyType,isModelSystem,isFeatured,ackContext,studyMetadata,accessReqs,isReleased,newsRelease,metadataType,Component',
  'Source': 'Sage Bionetworks',
  'Ontology': 'Sage Bionetworks'}]

In [55]:
study_temp = pd.concat([study_temp, pd.DataFrame(new_manifest)], axis = 0).reset_index(drop = True)

In [56]:
study_temp.loc[study_temp["Attribute"] == "metadataType", 'Valid Values'] = 'study'

In [61]:
study_temp.loc[study_temp["Attribute"] == "Species", 'Attribute'] = 'species'

In [57]:
study_temp["Valid Values"] = np.select(
    [
        study_temp["Attribute"] == "program",
        study_temp["Attribute"] == "projectShortName",
        study_temp["Attribute"] == "isFeatured",
        study_temp["Attribute"] == "isReleased",
        study_temp["Attribute"] == "Species",
        study_temp["Attribute"] == "isModelSystem",
    ],
    ["ELITE,AD", "LC,ILO,LLFS,LG", "TRUE,FALSE", "TRUE,FALSE", "Human,Mouse", 'TRUE,FALSE'],
    default=None,
)
# study_temp["Valid Values"]

0               None
1               None
2     LC,ILO,LLFS,LG
3               None
4               None
5        Human,Mouse
6               None
7               None
8               None
9               None
10          ELITE,AD
11              None
12              None
13              None
14              None
15              None
16              None
17        TRUE,FALSE
18        TRUE,FALSE
19              None
20              None
21              None
22        TRUE,FALSE
23              None
24              None
25              None
Name: Valid Values, dtype: object

In [58]:
study_temp = study_temp[dm_cols]

In [62]:
study_temp.to_csv(Path(ROOT_DIR, "data/rfc_tables_cleaned/study_RFC.csv"), index=False)

In [63]:
dm

,Attribute,Description,Valid Values,DependsOn,DependsOn Component,Required,Parent,Validation Rules,Properties,Source,Module,multivalue,columnType,Ontology,Notes,UsedIn
0,acquisitionBatchID,"Acquisition batch identifier, provided by the ...",NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
1,acquisitionBatchSize,The number of samples,NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
2,acquisitionBatchSizeUnit,The unit of measurement for number of samples ...,NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
3,acquisitionMode,The specific aspect of a mass spectrometer met...,NaN,NaN,NaN,True,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,https://www.ebi.ac.uk/ols/ontologies/ms/terms?...,A data contributor should be able to write in ...,Metabolomics
4,acquisitionSoftware,The name of the acquisition software used. Pro...,NaN,NaN,NaN,True,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,http://purl.obolibrary.org/obo/MS_1001455,A data contributor should be able to write in ...,Metabolomics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,visitCode,Indicate which longitudinal visit for the indi...,NaN,NaN,NaN,True,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,NUMBER,Sage Bionetworks,,Individual_Human
191,Whole_Genome_Sequencing,Template for Whole_Genome_Sequencing,NaN,"Filename,specimenID,sampleType,measurementTech...",NaN,False,Component,NaN,Template,NaN,Template,NaN,NaN,Sage Bionetworks,,NaN
192,rnaBatchSize,The number of samples,NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,NUMBER,Sage Bionetworks,,scRNAseq
193,rnaBatchSizeUnit,The unit of measurement for number of samples ...,"AFU,AI,AU/ml,bp g/dl,DK units/ml,g/l,gm,HAU,IU...",NaN,NaN,True,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,scRNAseq


In [67]:
additions = study_temp.loc[study_temp['Attribute'].isin(list(set(study_temp['Attribute']) - set(dm['Attribute'])))]

In [70]:
dm_new = pd.concat([dm, additions]).reset_index(drop = True)

In [71]:
dm_new

,Attribute,Description,Valid Values,DependsOn,DependsOn Component,Required,Parent,Validation Rules,Properties,Source,Module,multivalue,columnType,Ontology,Notes,UsedIn
0,acquisitionBatchID,"Acquisition batch identifier, provided by the ...",NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
1,acquisitionBatchSize,The number of samples,NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
2,acquisitionBatchSizeUnit,The unit of measurement for number of samples ...,NaN,NaN,NaN,False,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,Sage Bionetworks,,Metabolomics
3,acquisitionMode,The specific aspect of a mass spectrometer met...,NaN,NaN,NaN,True,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,https://www.ebi.ac.uk/ols/ontologies/ms/terms?...,A data contributor should be able to write in ...,Metabolomics
4,acquisitionSoftware,The name of the acquisition software used. Pro...,NaN,NaN,NaN,True,ManifestColumn,NaN,ManifestColumn,NaN,ManifestColumn,False,STRING,http://purl.obolibrary.org/obo/MS_1001455,A data contributor should be able to write in ...,Metabolomics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,accessReqs,None,None,None,None,True,ManifestColumn,None,ManifestColumn,Sage Bionetworks,ManifestColumn,False,STRING,Sage Bionetworks,None,study_folder_template
216,isReleased,None,"TRUE,FALSE",None,None,True,ManifestColumn,None,ManifestColumn,Sage Bionetworks,ManifestColumn,False,BOOLEAN,Sage Bionetworks,None,study_folder_template
217,newsRelease,None,None,None,None,True,ManifestColumn,None,ManifestColumn,Sage Bionetworks,ManifestColumn,False,STRING,Sage Bionetworks,None,study_folder_template
218,metadataType,None,None,None,None,True,ManifestColumn,None,ManifestColumn,Sage Bionetworks,ManifestColumn,False,STRING,Sage Bionetworks,None,study_folder_template


In [73]:
dm_new.to_csv(Path(ROOT_DIR, "EL.data.model.csv"))

In [75]:
syn.get("syn58352475", downloadFile = False)

File(species=['Mus musculus (mouse)'], DOI=['10.1038/S42003-023-05128-Y'], downloadLink=['ftp://ftp.pride.ebi.ac.uk/pride/data/archive/2023/07/PXD035255/031519-MouseLiver-26-GC2191-01.raw'], createdBy='3466893', dataRepository=['PRIDE'], _file_handle={'id': '139500629', 'etag': 'b711bde1-51b9-4fc2-b8c5-f01666c4a2ff', 'createdBy': '3466893', 'createdOn': '2024-04-22T20:20:43.000Z', 'modifiedOn': '2024-04-22T20:20:43.000Z', 'concreteType': 'org.sagebionetworks.repo.model.file.ExternalFileHandle', 'contentType': 'NOT_SET', 'fileName': '031519-MouseLiver-26-GC2191-01.raw', 'status': 'AVAILABLE', 'externalURL': 'ftp://ftp.pride.ebi.ac.uk/pride/data/archive/2023/07/PXD035255/031519-MouseLiver-26-GC2191-01.raw', 'contentSize': None, 'contentMd5': None, 'storageLocationId': None}, id='syn58352475', dataCoordinationCenter=['ELITE'], parentId='syn58351053', name='031519-MouseLiver-26-GC2191-01.raw', cacheDir=None, fileName=['031519-MouseLiver-26-GC2191-01.raw'], forceVersion=[False], versionNumb

In [81]:
query = syn.tableQuery(
    """ SELECT "dataFileKey" FROM syn52234677 WHERE study = 'PXD035255'"""
)
query = query.asDataFrame()

query["dataFileKey"].to_csv("ftp-test.csv", index=False)